# 设计自定义层
这一节主要介绍如何使用底层的 NDArray 借口来实现一个 Gluon 层，从而可以被后面重读调用

## 定义一个简单的层
首先看一下如何定义一个简单的层，他不需要维护模型参数。下面代码定义一个层将输入值减去均值

In [2]:
from mxnet import nd
from mxnet.gluon import nn

class CenteredLayers(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayers, self).__init__(**kwargs)
        
    def forward(self, x):
        return x - x.mean()

layer = CenteredLayers()
layer(nd.array([1, 2, 3, 4,5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [5]:
# 可以用来构造更加复杂的层神经网路

net = nn.Sequential()
with net.name_scope():
    net.add(
        nn.Dense(128),
        nn.Dense(10),
        CenteredLayers()
    )

# 确认输出的均值确实是 0
net.initialize()
y = net(nd.random_uniform(shape=(4, 8)))
y.mean






<bound method NDArray.mean of 
[[ 0.01346263 -0.05141733  0.0258197  -0.0008087   0.03806411  0.02332325
   0.011646   -0.00327658 -0.02827016 -0.00022905]
 [ 0.02151005 -0.056457    0.02358621 -0.0124314   0.0187663   0.02040836
   0.02771828  0.02639681 -0.04175982 -0.02731595]
 [ 0.00828139 -0.04330623  0.0335821  -0.03011171  0.00255397  0.00301897
   0.0281186   0.01753271 -0.03377977 -0.03513451]
 [ 0.02541625 -0.02574518  0.02376265  0.00475178  0.00461294  0.020955
   0.02122782  0.00244135 -0.03230387 -0.02461   ]]
<NDArray 4x10 @cpu(0)>>

# 带模型参数的自定义层
虽然 CenteredLayer 可能会告诉实现自定义层大概是什么样子，但是它缺少了给场重要的一块，就是他没有可以学习的模型参数。

之前访问 Dense 层的权重的时候是通过 dense.weight.data(),这里 weight 是一个 Parameter 的类型。我们可以显示的构建这样一个参数。

In [6]:
from mxnet import gluon
my_param = gluon.Parameter("exciting_parameter_yay", 
                           shape=(3,3))


In [7]:
# 穿件一个 3*3 大小的参数并取名为 “exicting_parameter_yay" 
# 然后用默认方法初始化打印结果

my_param.initialize()
(my_param.data(), my_param.grad())

(
 [[-0.04531868 -0.03886211  0.05232579]
  [ 0.05935124  0.0243485   0.03545631]
  [ 0.06520855  0.00830095  0.00880194]]
 <NDArray 3x3 @cpu(0)>, 
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 <NDArray 3x3 @cpu(0)>)

通常自定义层的时候我们不会直接创建Parameter，而是用过Block自带的一个ParamterDict类型的成员变量params，顾名思义，这是一个由字符串名字映射到Parameter的字典。

In [9]:
pd = gluon.ParameterDict(prefix="block1_")
pd.get("exicting_parameter_yay")
pd



KeyError: 'shape'

In [10]:
class MyDense(nn.Block):
    def __init__(self, uints, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        with self.name_scope():
            self.weight = self.params.get(
                'weight', shape=(in_units,uints)
            )
            self.bias = self.params.get('bias', 
                                        shape=(uints,))
            
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + \
                    self.bias.data()
        return nd.relu(linear)    